In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img  

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

# New Section

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
base_dir="//content//drive//My Drive//Hackathon data"


In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range = 40, 
        shear_range = 0.2, 
        zoom_range = 0.2, 
        horizontal_flip = True, 
        brightness_range = (0.5, 1.5))

In [0]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')



Found 339 images belonging to 10 classes.


In [0]:
img = load_img('/content/drive/My Drive/Hackathon data/Mustard/download (1).jpg')

In [0]:
x = img_to_array(img)

In [0]:
x = x.reshape((1, ) + x.shape)

In [0]:
i = 0
for batch in datagen.flow(x, batch_size = 1, 
                          save_to_dir ='/content/drive/My Drive/Hackathon data/Mustard',  
                          save_prefix ='image', save_format ='jpeg'): 
    i += 1
    if i > 20: 
        break

In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img  

Using TensorFlow backend.


In [0]:
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 81 images belonging to 10 classes.


In [0]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 10))

In [0]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Barley': 0, 'Corn': 1, 'Millet': 2, 'Mustard': 3, 'Sorghum': 4, 'olive': 5, 'rice': 6, 'soyabean': 7, 'sugarcane': 8, 'wheat': 9}


In [0]:
!cat labels.txt

Barley
Corn
Millet
Mustard
Sorghum
olive
rice
soyabean
sugarcane
wheat

In [0]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
base_model.trainable = False

In [0]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 2,626,986
Trainable params: 369,002
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
epochs = 20

history = model.fit_generator(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 6 steps, validate for 2 steps
Epoch 1/20
6/6 [==============================] - 67s 11s/step - loss: 2.4376 - accuracy: 0.2006 - val_loss: 2.1150 - val_accuracy: 0.1852
Epoch 2/20
6/6 [==============================] - 8s 1s/step - loss: 1.7113 - accuracy: 0.2861 - val_loss: 1.8898 - val_accuracy: 0.2222
Epoch 3/20
6/6 [==============================] - 8s 1s/step - loss: 1.5715 - accuracy: 0.4218 - val_loss: 1.9231 - val_accuracy: 0.2963
Epoch 4/20
6/6 [==============================] - 8s 1s/step - loss: 1.4711 - accuracy: 0.4661 - val_loss: 1.9143 - val_accuracy: 0.3580
Epoch 5/20
6/6 [==============================] - 8s 1s/step - loss: 1.2838 - accuracy: 0.5782 - val_loss: 1.8361 - val_accuracy: 0.4198
Epoch 6/20
6/6 [==============================] - 8s 1s/step - loss: 1.1735 - accuracy: 0.5929 - val_loss: 1.8237 - val_accuracy: 0.4444
Epoch 7/20

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

NameError: ignored

In [0]:
base_model.trainable = True

In [0]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  155


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 2,626,986
Trainable params: 2,231,594
Non-trainable params: 395,392
_________________________________________________________________


In [0]:
history_fine = model.fit_generator(train_generator, 
                         epochs=5,
                         validation_data=val_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 4 steps, validate for 1 steps
Epoch 1/5
4/4 [==============================] - 7s 2s/step - loss: 0.6357 - accuracy: 0.8275 - val_loss: 1.7663 - val_accuracy: 0.4333
Epoch 2/5
4/4 [==============================] - 3s 739ms/step - loss: 0.5644 - accuracy: 0.8627 - val_loss: 1.7174 - val_accuracy: 0.4333
Epoch 3/5
4/4 [==============================] - 3s 733ms/step - loss: 0.5095 - accuracy: 0.8980 - val_loss: 1.6775 - val_accuracy: 0.4333
Epoch 4/5
4/4 [==============================] - 3s 742ms/step - loss: 0.4708 - accuracy: 0.9059 - val_loss: 1.6457 - val_accuracy: 0.4667
Epoch 5/5
4/4 [==============================] - 3s 734ms/step - loss: 0.4355 - accuracy: 0.9176 - val_loss: 1.6181 - val_accuracy: 0.4667


In [0]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: save/fine_tuning/assets


In [0]:
from google.colab import files

files.download('/content/labels.txt')
files.download('/content/model.tflite')

In [0]:

import matplotlib.pyplot as plt

AssertionError: ignored